# FLIP (04): Learning Theory (I)
**(Module 03: Operation Research)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

# Solving Linear Programming Problems with PuLP

## Installing PuLP

PuLP is an external package that may need to be installed into your python environment. The following cell issues a system command to install PuLP if needed. PuLP includes the [COIN-OR CBC](https://projects.coin-or.org/Cbc) solver.

In [ ]:
!pip install PuLP

## Example: Giapetto's Workshop

### Model Formulation

In [ ]:
import pulp

# Create problem instance
giapetto = pulp.LpProblem("Giapetto's Workshop", pulp.LpMaximize)

# Define Variables with lower bounds of zero
x1 = pulp.LpVariable("Soldiers",0)
x2 = pulp.LpVariable("Trains",0)

# Add Objective
giapetto += 3*x1 + 2*x2, "Profit"

# Add Constraints
giapetto += 2*x1 + x2 <= 100,"Finishing Labor"
giapetto += x1 + x2 <= 80, "Carpentry Labor"
giapetto += x1 <= 40, "Soldier Demand"
giapetto += x1 + x2 == 20, "Minimum Production"

# Display Problem
print giapetto

### Model Solution

In [ ]:
giapetto.solve()

print pulp.LpStatus[giapetto.status]
print pulp.LpSenses[giapetto.sense], giapetto.objective.name, "=", pulp.value(giapetto.objective)

### Displaying the Decision Variables

Values for the decision variables can be accessed with the `pulp.value()` method.

In [ ]:
print x1.name, "=" , pulp.value(x1)
print x2.name, "=" , pulp.value(x2)

Alternatively, the decision variables can be accessed by interating through the list created by the `variables()` method.

In [ ]:
for x in giapetto.variables():
    print "Name:       ", x.name
    print "Value:      ", x.varValue
    print "Category:   ", x.cat
    print "Lower Bound:", x.lowBound
    print "Upper Bound:", x.upBound
    print 

[Pandas](http://pandas.pydata.org/) is a python package for managing and analysing data. The following cell shows how to use a list comprehension and panda DataFrame to create and display a table from the solution data.

### Displaying the Constraints

The constraints are stored as an ordered dictionary in `constraints` attribute of the problem. 

In [ ]:
for (name,constraint) in giapetto.constraints.items():
    print "Name:        ", name
    print "Constraint:  ", constraint
    print "Lower Bound: ", constraint.getLb()
    print "Upper Bound: ", constraint.getUb()
    print "Value:       ", pulp.value(constraint)
    print "Sense:       ", pulp.LpConstraintSenses[constraint.sense]
    print "Constant:    ", constraint.constant
    print "Slack:       ", constraint.slack
    print "Slack (Feas):", constraint.slack if constraint.sense < 0 else -constraint.slack
    print

## A simple function to solve and display the solution to a linear program

In [ ]:
import pandas

def solvelp(lp):
    lp.solve();

    data = [[lp.objective.name, pulp.LpSenses[lp.sense], pulp.LpStatus[lp.status], pulp.value(lp.objective)]]
    objDF = pandas.DataFrame(data,columns=["Objective", "Sense", "Status", "Value"])

    data = [[x.name, x.cat, x.lowBound, x.upBound, x.varValue] 
        for x in lp.variables()]
    varDF = pandas.DataFrame(data,columns=["Name","Category","Lower Bound","Upper Bound","Value"])    

    data = [[c.name, c.getLb(), c.getUb(), c.slack, pulp.value(c) - c.constant] 
        for (k,c) in lp.constraints.items()]
    conDF = pandas.DataFrame(data,columns=["Name", "Lower Bound", "Upper Bound", "Slack", "Value"])

    return (objDF,varDF,conDF)

In [ ]:
(odf,vdf,cdf) = solvelp(giapetto);

from IPython.display import display,HTML

display(HTML('<h3>Model</h3>'))
display(giapetto)
display(HTML('<h3>Objective</h3>'))
display(odf)
display(HTML('<h3>Variables</h3>'))
display(vdf)
display(HTML('<h3>Constraints</h3>'))
display(cdf)

In [ ]:
cdf.plot(title="Value",kind='bar')